# 1. Import libraries

In [ ]:
# data manipulation and plotting tools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# data processing
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler,LabelEncoder

# algorithms
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# model selection tools
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

# metrics
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

# explainability
import shap

#visual
import seaborn as sns

from scipy.stats import chi2_contingency

#MostrarMas
pd.set_option('display.max_columns', None)

# 2. Load the dataset

In [ ]:
data = pd.read_csv('churn.csv')

In [ ]:
data.head()


In [ ]:
data.info()

# 3. Data Splitting

In [ ]:
#X_trainin, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Exploratory Data Analysis

In [ ]:
# Your code here. Exploratory data analysis can be done before splitting, but do not transform the dataset before splitting.
data.info() #No valores nulos 


In [ ]:
missing_percentage_Offer = (data['Offer'].isnull().sum() / len(data)) * 100
print(f"Porcentaje de valores faltantes en la columna 'Offer': {missing_percentage_Offer:.2f}%")

In [ ]:
missing_percentage_Internet_Type = (data['Internet Type'].isnull().sum() / len(data)) * 100
print(f"Porcentaje de valores faltantes en la columna 'Internet Type': {missing_percentage_Internet_Type:.2f}%")

In [ ]:
#Estructura de los datos y resumen:  El dataset original tiene 50 columnas,
#con variables de datos demográficos, geográficas, con información sobre los servicios,
#datos financieros, y satisfacción y comportamiento del cliente.
#Análisis de datos faltantes: Las siguientes columnas tienen valores faltantes:  Offer (55,05%)  e Internet Type (21,67%) 
#Esta información está relacionada con los clientes que se han dado de baja, y se van a ir evaluando a medida que 
#se realice el estudio del dataset

#### Estudio edades

In [ ]:
mean_age = data['Age'].mean()
print(f"La media de la edad es: {mean_age}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['Age'], bins=20, kde=True, color='skyblue')
plt.title('Distribución de la Edad de los Clientes')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
mean_age = data['Age'].mean()
print(f"La media de la edad es: {mean_age}")

In [ ]:
#Los rangos de edad están desde los 19 a 80 años, siendo la media alrededor de los 46.50 años

In [ ]:
sns.histplot(data['Tenure in Months'])
plt.show()

In [ ]:
data['Tenure in Months'].describe()

In [ ]:
sns.histplot(data["Number of Dependents"])
plt.show()


In [ ]:
data["Multiple Lines"].value_counts(normalize=True)

In [ ]:
# Crear tabla de contingencia
tabla_contingencia_multiple_lines = pd.crosstab(data['Multiple Lines'], data['Churn Label'])

# Aplicar la prueba Chi-cuadrado
chi2_multiple_lines, p_multiple_lines, dof_multiple_lines, expected_multiple_lines = chi2_contingency(tabla_contingencia_multiple_lines)

# Calcular el índice de Cramér
n_multiple_lines = data.shape[0]  # número de observaciones
cramer_v_multiple_lines = np.sqrt(chi2_multiple_lines / (n_multiple_lines * (min(tabla_contingencia_multiple_lines.shape) - 1)))

# Mostrar resultados
print(f"Chi-cuadrado: {chi2_multiple_lines}")
print(f"Valor p: {p_multiple_lines}")
print(f"Índice de Cramér: {cramer_v_multiple_lines}")


In [ ]:
data["Satisfaction Score"].value_counts()

In [ ]:
data.groupby("Satisfaction Score")["Churn Label"].value_counts(normalize=True)

##Relevante, vale la pena hacer feature Engineering

In [ ]:
data["Customer Status"].value_counts

In [ ]:
data["Customer Status"].value_counts(normalize="True")

In [ ]:
data["Churn Label"].value_counts(normalize="True")

In [ ]:
data["Payment Method"].value_counts(normalize=True) ##Mas de la mitad son por Bank

In [ ]:
# Crear el DataFrame a partir de tus datos
data_frame = {'Payment Method': ['Bank Withdrawal', 'Bank Withdrawal', 'Credit Card', 'Credit Card', 'Mailed Check', 'Mailed Check'],
        'Churn Label': ['No', 'Yes', 'No', 'Yes', 'No', 'Yes'],
        'Proportion': [0.660015, 0.339985, 0.855220, 0.144780, 0.631169, 0.368831]}

df = pd.DataFrame(data_frame)

# Pivotar la tabla para que Churn Label sea la columna
pivot_df = df.pivot(index='Payment Method', columns='Churn Label', values='Proportion')

# Graficar barras apiladas
pivot_df.plot(kind='bar', stacked=True)

plt.title('Proporción de Churn por Método de Pago')
plt.ylabel('Proporción')
plt.xticks(rotation=45)
plt.show()

In [ ]:
data[(data["Total Refunds"]!=0) & (data["Churn Label"]=="Yes")] ## Ver cuan relacionado esta que haya reembolsos con el churn

In [ ]:
value_counts = data["Total Extra Data Charges"].value_counts()

# Convierte el resultado de value_counts a un DataFrame para facilitar la visualización
df = value_counts.reset_index()
df.columns = ['Total Extra Data Charges', 'Count']
df

#### Estudio breve 

In [ ]:
plt.figure(figsize=(15, 6))  # Ajusta el valor 15 para hacerla aún más ancha si lo necesitas
plt.bar(df['Total Extra Data Charges'], df['Count'], color='skyblue')
plt.xlabel('Total Extra Data Charges')
plt.ylabel('Count')
plt.title('Distribution of Total Extra Data Charges')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
sns.histplot(data["Total Extra Data Charges"], bins=140)
plt.show()

In [ ]:
sns.histplot(data['Total Charges'])
plt.show()

In [ ]:
sns.histplot(data['Total Revenue'])
plt.show()

In [ ]:
data["Referred a Friend"].value_counts()

In [ ]:
sns.histplot(data['Satisfaction Score'],bins = 5) ## por lo general es una satisfaccion alta
plt.show()

In [ ]:
data["Premium Tech Support"].value_counts()

In [ ]:
contingency_table = pd.crosstab(data['Premium Tech Support'], data['Churn Label'])
print(contingency_table)

In [ ]:
data['Country'].value_counts()

In [ ]:
columns_to_analyze = [
    'Offer', 
    'Phone Service', 
    'Avg Monthly Long Distance Charges', 
    'Multiple Lines', 
    'Internet Service', 
    'Avg Monthly GB Download', 
    'Online Security'
]

In [ ]:
filtered_data = data[columns_to_analyze]

In [ ]:
filtered_data_summary = filtered_data.describe(include='all')

filtered_data_summary

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
# 1. Gráfico de barras para Offer y Phone Service
sns.countplot(ax=axes[0, 0], data=filtered_data, x='Offer', hue='Phone Service')
axes[0, 0].set_title('Distribución de Ofertas según el Servicio de Teléfono')
axes[0, 0].set_ylabel('Cantidad de clientes')
axes[0, 0].set_xlabel('Oferta')

# 2. Gráfico de violín para Internet Service y Avg Monthly GB Download
sns.violinplot(ax=axes[0, 1], data=filtered_data, x='Internet Service', y='Avg Monthly GB Download')
axes[0, 1].set_title('Relación entre Servicio de Internet y GB Descargados')
axes[0, 1].set_ylabel('Promedio de GB descargados mensualmente')
axes[0, 1].set_xlabel('Servicio de internet')

# 3. Gráfico de dispersión para Avg Monthly Long Distance Charges y Multiple Lines
sns.boxplot(ax=axes[1, 0], data=filtered_data, x='Multiple Lines', y='Avg Monthly Long Distance Charges')
axes[1, 0].set_title('Distribución de cargos larga distancia por múltiples líneas')
axes[1, 0].set_ylabel('Cargos mensuales por larga distancia')
axes[1, 0].set_xlabel('Múltiples líneas')

# 4. Gráfico de caja para Online Security y Avg Monthly GB Download
sns.boxplot(ax=axes[1, 1], data=filtered_data, x='Online Security', y='Avg Monthly GB Download')
axes[1, 1].set_title('Relación entre Seguridad en Línea y GB Descargados')
axes[1, 1].set_ylabel('Promedio de GB descargados mensualmente')
axes[1, 1].set_xlabel('Seguridad en línea')

plt.tight_layout()
plt.show()

In [ ]:
# 1
# Distribución de Ofertas según el Servicio de Teléfono: 
# La mayoría de los clientes no tienen ninguna oferta activa ("None")
# para quienes tienen servicio de teléfono como para los que no lo tienen
# los clientes con servicio de teléfono son más en todas las categorías de oferta
# 2
# Relación entre el Servicio de Internet y los GB Descargados: 
# Los clientes con servicio de internet muestran una amplia distribución en la cantidad de GB descargados por mes
# la mediana está en 17 GB
# los que no tienen servicio de internet muestran valores cercanos a 0 GB
# 3
# Distribución de Cargos de Larga Distancia por Múltiples Líneas: 
# Los clientes con múltiples líneas suelen tener cargos de larga distancia un poco más altos 
# comparando con los que no tienen múltiples líneas
# los cargos están bien distribuidos en las categorías
# 4
# Relación entre Seguridad en Línea y GB Descargados: 
# Los clientes que no tienen seguridad en línea tienden a descargar más GB mensualmente en comparación con los que sí 
# los clientes que buscan mayor seguridad en línea no utilizan tanto la capacidad de internet

#### Estudio de correlacion

In [ ]:
numerical_columns = ['Age', 'Number of Dependents',
       'Population', 'Number of Referrals', 'Tenure in Months',
       'Avg Monthly Long Distance Charges', 'Avg Monthly GB Download',
       'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Satisfaction Score',
       ]

In [ ]:
#corr calcula coeficiente de correlacion de pearson, y solo toma las numericas
sns.heatmap(data[numerical_columns].corr(), annot=True, fmt=".2f")

# 5. Data Processing

## Data Cleaning

In [ ]:
# Your code here
data.info()
ColumnsDrop=["Customer ID","Under 30","Senior Citizen","Country","State","City","Latitude","Longitude","Quarter","Churn Category","Churn Score","CLTV","Churn Reason","Customer Status"]

In [ ]:
data = data.drop(ColumnsDrop,axis=1)

In [ ]:
data.info()

In [ ]:
data["Contract"].value_counts()

In [ ]:
 ## Limpiar los datos Internet Type, offer
data["Internet Type"].value_counts()

In [ ]:
data["Internet Type"].isnull().sum()

In [ ]:
data[(data["Internet Type"].isnull())&((data["Internet Service"]==0))]

In [ ]:
data['Internet Type'] = data['Internet Type'].fillna("No Internet")
data["Internet Type"].isnull().sum()

In [ ]:
data["Offer"].value_counts()

In [ ]:
data["Offer"].isnull().sum()

In [ ]:
data['Offer'] = data['Offer'].fillna("No Offer")

In [ ]:
data["Offer"].isnull().sum()

In [ ]:
data.info()## No nulos

## Re-sampling (if needed)

In [ ]:

data["Churn Label"].value_counts(normalize=True)
##No necesario el re-sampling
## Hay suficientos datos para que el programa aprenda


## Feature Engineering

In [ ]:
# Your code here
data["Age_cuartiles"] = pd.qcut(data['Age'], q=4)
data["Age_cuartiles"].value_counts()

In [ ]:
# Agrupar por los cuartiles de edad y contar los valores normalizados (proporciones) de "Churn Label"
proporciones = data.groupby("Age_cuartiles")["Churn Label"].value_counts(normalize=True)

# Mostrar el resultado
print(proporciones)

## Los grupos mayores son mas propensos a caer en churn

In [ ]:
# Crear una nueva columna categórica para los niveles de satisfacción
data['Satisfaction_Level'] = pd.cut(data['Satisfaction Score'], 
                                    bins=[0, 2, 3, 5],  # Límites de los grupos
                                    labels=['Baja', 'Media', 'Alta'])  # Nombres de los grupos

# Ver el resultado agrupado
print(data.groupby('Satisfaction_Level')["Churn Label"].value_counts(normalize=True))

In [ ]:
# Crear una tabla de contingencia entre los niveles de satisfacción y el churn
tabla_contingencia_satisfaction = pd.crosstab(data['Satisfaction_Level'], data['Churn Label'])

# Aplicar la prueba Chi-cuadrado
chi2_satisfaction, p_satisfaction, dof_satisfaction, expected_satisfaction = chi2_contingency(tabla_contingencia_satisfaction)

# Calcular el índice de Cramér
n_satisfaction = data.shape[0]  # número de observaciones
cramer_v_satisfaction = np.sqrt(chi2_satisfaction / (n_satisfaction * (min(tabla_contingencia_satisfaction.shape) - 1)))

# Mostrar resultados
print(f"Chi-cuadrado: {chi2_satisfaction}")
print(f"Valor p: {p_satisfaction}")
print(f"Índice de Cramér: {cramer_v_satisfaction}")


In [ ]:
##ANALISIS DE CORRELACION ENTRE SATISFACTION LEVEL Y CHURN LABEL

## Encoding

In [ ]:
labelEncoderColumns = ["Paperless Billing","Churn Label","Gender","Married","Dependents","Referred a Friend","Phone Service","Multiple Lines","Internet Service","Online Security","Online Backup","Device Protection Plan","Premium Tech Support","Streaming TV","Streaming Movies","Streaming Music","Unlimited Data"]
OneHotEncoderColumns = ["Offer","Internet Type","Contract","Payment Method"]
OrdinalEncoderColumns = ["Age_cuartiles","Satisfaction_Level"]
#Offer,Internet Type,Contract,Payment Method


In [ ]:
label_encoder = LabelEncoder()

In [ ]:
for col in labelEncoderColumns:
    data[col] = label_encoder.fit_transform(data[col])


In [ ]:
print(label_encoder.classes_)

In [ ]:
data.info()

In [ ]:
oh_encoder = OneHotEncoder(cols = OneHotEncoderColumns, use_cat_names=True)

In [ ]:
oh_encoder.fit(data)

In [ ]:
data = oh_encoder.transform(data)

In [ ]:
data.columns

In [ ]:
data.info()

columnas = ["Gender","Age","Married","Number of Dependents","Referred a Friend","Tenure in Months"]

columnasModelo = ['Gender', 'Age', 'Married', 'Number of Dependents',
        'Referred a Friend',
       'Tenure in Months', 'Offer_No Offer', 'Offer_Offer E', 'Offer_Offer D',
       'Offer_Offer C', 'Offer_Offer B', 'Offer_Offer A', 'Phone Service',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type_DSL', 'Internet Type_Fiber Optic',
       'Internet Type_Cable', 'Internet Type_No Internet',
       'Avg Monthly GB Download', 'Online Security', 'Online Backup',
       'Device Protection Plan', 'Premium Tech Support', 'Streaming TV',
       'Streaming Movies', 'Streaming Music', 'Unlimited Data',
       'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year',
       'Paperless Billing', 'Payment Method_Bank Withdrawal',
       'Payment Method_Credit Card', 'Payment Method_Mailed Check',
       'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Satisfaction Score', 'Churn Label']

In [ ]:
data["Satisfaction_Level"].value_counts()

In [ ]:
ordinal_encoder = OrdinalEncoder(
    cols=OrdinalEncoderColumns,
    mapping=[
        {
            "col": "Age_cuartiles",
            "mapping": {"(18.999, 32.0]": 0, "(32.0, 46.0]": 1, "(46.0, 60.0]": 2, "(60.0, 80.0]": 3}
        },
        {
            "col": "Satisfaction_Level",
            "mapping": {"Baja": 0, "Media": 1, "Alta": 2}
        }
    ]
)

In [ ]:
ordinal_encoder = ordinal_encoder.fit(X=data)
data= ordinal_encoder.transform(X=data)

In [ ]:
data.info()

## Scaling (if needed)

In [ ]:
scaler = RobustScaler()

In [ ]:
data_escalada = data.copy()
data_escalada

In [ ]:
data_escalada = scaler.fit_transform(data_escalada)

In [ ]:
data_escalada = pd.DataFrame(data_escalada, columns=data.columns)

In [ ]:
data_escalada

### DATA SPLITING


In [ ]:
columnasModelo = ['Gender', 'Age_cuartiles', 'Married', 'Number of Dependents',
        'Referred a Friend', 'Tenure in Months', 'Offer_No Offer', 'Offer_Offer E', 'Offer_Offer D',
       'Offer_Offer C', 'Offer_Offer B', 'Offer_Offer A', 'Phone Service',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type_DSL', 'Internet Type_Fiber Optic',
       'Internet Type_Cable', 'Internet Type_No Internet',
       'Avg Monthly GB Download', 'Online Security', 'Online Backup',
       'Device Protection Plan', 'Premium Tech Support', 'Streaming TV',
       'Streaming Movies', 'Streaming Music', 'Unlimited Data',
       'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year',
       'Paperless Billing', 'Payment Method_Bank Withdrawal',
       'Payment Method_Credit Card', 'Payment Method_Mailed Check',
       'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
        'Satisfaction_Level']

In [ ]:
train_data, tmp_data = train_test_split(data_escalada, test_size = 0.30,stratify = data_escalada["Churn Label"],random_state = 1)
val_data, test_data = train_test_split(tmp_data, test_size = 0.50,stratify = tmp_data["Churn Label"],random_state = 1)


In [ ]:
len(data_escalada),len(train_data),len(val_data),len(test_data)

In [ ]:
print(train_data.columns)

In [ ]:
#columnasModelo = [col for col in columnasModelo if col in train_data.columns]


## Decision Tree Classifier

In [ ]:
#TO DO SORAYA

In [ ]:
#model = DecisionTreeClassifier()

In [ ]:
#model.fit(X=X,y=data["Churn Label"])

In [ ]:
#plt.figure(figsize=(7, 7))
#plot_tree(model, feature_names=X.columns, class_names=["no churn", "churn"])

In [ ]:
#alta pureza en nuestro arbol de decision. Cada hoja alcanza a tener cero.

In [ ]:
#predicted_prob = model.predict_proba(X)

In [ ]:
#predicted_prob

In [ ]:
#predicted_class = model.predict(X)

In [ ]:
#predicted_class

In [ ]:
#data_and_predictions = data.copy()
#data_and_predictions

In [ ]:
#data_and_predictions["predicted_class"] = predicted_class
#data_and_predictions["predicted_prob(no churn)"] = predicted_prob[:,0]
#data_and_predictions["predicted_prob(churn)"] = predicted_prob[:,1]

In [ ]:
#def calculate_accuracy(predictions):
#    number_of_hits = (predictions["Churn Label"] == predictions["predicted_class"]).sum()
#    number_of_predictions = len(predictions)

#    accuracy = round(number_of_hits/number_of_predictions*100, 2)
#    accuracy = float(accuracy)

 #   return accuracy

In [ ]:
#calculate_accuracy(data_and_predictions)

In [ ]:
#data

In [ ]:
#X_train.info()

## KNN 

#### SIN GRID Y COLUMNAS FILTRADAS


In [ ]:
X_train = train_data.drop(columns=["Churn Label"])
y_train = train_data["Churn Label"]

X_val = val_data.drop(columns=["Churn Label"])
y_val = val_data["Churn Label"]

X_test = test_data.drop(columns=["Churn Label"])
y_test = test_data["Churn Label"]

In [ ]:
KNclassifier1 = KNeighborsClassifier(n_neighbors=20)

In [ ]:
KNclassifier1.fit(X_train[columnasModelo], y_train)

In [ ]:
train_metrics_kn = train_data.copy()
train_metrics_kn["Preciction_knn1"] = KNclassifier1.predict(train_data[columnasModelo]) 

val_metrics_kn = val_data.copy()
val_metrics_kn["Preciction_knn1"] = KNclassifier1.predict(val_data[columnasModelo])



In [ ]:
print("Training Metrics:")
print(classification_report(y_true=train_metrics_kn["Churn Label"], y_pred=train_metrics_kn["Preciction_knn1"]))

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=val_metrics_kn["Churn Label"], y_pred=val_metrics_kn["Preciction_knn1"]))

In [ ]:
cm = confusion_matrix(val_metrics_kn["Churn Label"], val_metrics_kn["Preciction_knn1"])
print(cm)
# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#### SIN GRID Y TODAS COLUMNAS

In [ ]:
KNclassifier2 = KNeighborsClassifier(n_neighbors=20)

In [ ]:
columnasSinChurn = ['Gender', 'Age', 'Married', 'Dependents', 'Number of Dependents',
       'Zip Code', 'Population', 'Referred a Friend', 'Number of Referrals',
       'Tenure in Months', 'Offer_No Offer', 'Offer_Offer E', 'Offer_Offer D',
       'Offer_Offer C', 'Offer_Offer B', 'Offer_Offer A', 'Phone Service',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type_DSL', 'Internet Type_Fiber Optic',
       'Internet Type_Cable', 'Internet Type_No Internet',
       'Avg Monthly GB Download', 'Online Security', 'Online Backup',
       'Device Protection Plan', 'Premium Tech Support', 'Streaming TV',
       'Streaming Movies', 'Streaming Music', 'Unlimited Data',
       'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year',
       'Paperless Billing', 'Payment Method_Bank Withdrawal',
       'Payment Method_Credit Card', 'Payment Method_Mailed Check',
       'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Satisfaction Score', 'Age_cuartiles',
       'Satisfaction_Level']

In [ ]:
KNclassifier2.fit(X = train_data[columnasSinChurn], y = train_data["Churn Label"])

In [ ]:
train_metrics_kn["Preciction_knn2"] = KNclassifier2.predict(train_data[columnasSinChurn]) 

val_metrics_kn["Preciction_knn2"] = KNclassifier2.predict(val_data[columnasSinChurn])



In [ ]:
print("Training Metrics:")
print(classification_report(y_true=train_metrics_kn["Churn Label"], y_pred=train_metrics_kn["Preciction_knn2"]))

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=val_metrics_kn["Churn Label"], y_pred=val_metrics_kn["Preciction_knn2"]))

In [ ]:
cm = confusion_matrix(val_metrics_kn["Churn Label"], val_metrics_kn["Preciction_knn2"])
print(cm)
# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#### CON GRID Y COLUMNAS FILTRADAS


In [ ]:
KNclassifierGrid = KNeighborsClassifier()

In [ ]:
param_grid = {
    'n_neighbors': [5, 10, 15, 20],  # Puedes ajustar este rango según sea necesario
    'weights': ['uniform', 'distance'],  # Probar pesos uniformes y basados en la distancia
    'metric': ['euclidean', 'manhattan']  # Probar diferentes métricas de distancia
}

In [ ]:
grid_search = GridSearchCV(
    estimator=KNclassifierGrid,
    param_grid=param_grid,
    cv=5, 
    scoring='accuracy')


In [ ]:
grid_search.fit(X = train_data[columnasModelo], y = train_data["Churn Label"])

In [ ]:
bestKnnGrid = grid_search.best_estimator_

In [ ]:
print("Mejores parámetros encontrados:", grid_search.best_params_)

In [ ]:
train_metrics_kn["Preciction_knn3"] = bestKnnGrid.predict(train_data[columnasModelo]) 

val_metrics_kn["Preciction_knn3"] = bestKnnGrid.predict(val_data[columnasModelo])

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=train_metrics_kn["Churn Label"], y_pred=train_metrics_kn["Preciction_knn3"]))

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=val_metrics_kn["Churn Label"], y_pred=val_metrics_kn["Preciction_knn3"]))

In [ ]:
cm = confusion_matrix(val_metrics_kn["Churn Label"], val_metrics_kn["Preciction_knn3"])
print(cm)
# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#### CON GRID Y COLUMNAS MÁS RELEVANTES

In [ ]:
columnasRelevantes = [
    'Gender',  # El género podría estar relacionado con patrones de comportamiento de uso.
    'Age',  # La edad puede influir en el tipo de servicios contratados y la retención.
    'Married',  # Estado civil, tal vez relacionado con la estabilidad del cliente.
    'Tenure in Months',  # Antigüedad con la empresa, un factor clave para el churn.
    'Phone Service',  # Tener o no servicio telefónico puede ser un indicador de compromiso.
    'Avg Monthly Long Distance Charges',  # Cargos mensuales de larga distancia, podrían mostrar la dependencia del servicio.
    'Multiple Lines',  # Líneas múltiples pueden indicar mayor compromiso con los servicios.
    'Internet Service',  # Servicio de Internet como factor principal de retención.
    'Avg Monthly GB Download',  # Uso de datos, relacionado con la intensidad del uso de los servicios.
    'Online Security',  # Servicios adicionales que pueden influir en la lealtad del cliente.
    'Streaming TV',  # Servicios de entretenimiento también pueden afectar la retención.
    'Unlimited Data',  # Servicios de datos ilimitados pueden ser una ventaja competitiva.
    'Contract_Month-to-Month',  # Clientes con contratos mensuales suelen tener mayor probabilidad de irse.
    'Paperless Billing',  # Preferencia por facturación sin papel puede indicar un cliente más comprometido digitalmente.
    'Payment Method_Bank Withdrawal',  # Método de pago, algunos métodos pueden asociarse a mayor estabilidad.
    'Monthly Charge',  # El monto mensual podría ser un indicador de carga financiera.
    'Satisfaction Score',  # La satisfacción es un predictor directo de churn.
    'Churn Label'  # La variable objetivo.
]

In [ ]:
KNclassifierGrid2 = KNeighborsClassifier()

In [ ]:
grid_search = GridSearchCV(estimator=KNclassifierGrid2, param_grid=param_grid, cv=5, scoring='accuracy')


In [ ]:
grid_search.fit(X = train_data[columnasRelevantes], y = train_data["Churn Label"])

In [ ]:
bestKnnGrid = grid_search.best_estimator_

In [ ]:
print("Mejores parámetros encontrados:", grid_search.best_params_)

In [ ]:
train_metrics_kn["Preciction_knn4"] = bestKnnGrid.predict(train_data[columnasRelevantes]) 

val_metrics_kn["Preciction_knn4"] = bestKnnGrid.predict(val_data[columnasRelevantes])

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=train_metrics_kn["Churn Label"], y_pred=train_metrics_kn["Preciction_knn4"]))

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=val_metrics_kn["Churn Label"], y_pred=val_metrics_kn["Preciction_knn4"]))

In [ ]:
cm = confusion_matrix(val_metrics_kn["Churn Label"], val_metrics_kn["Preciction_knn4"])
print(cm)
# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#### Verificar con el test

In [ ]:
y_pred_test = bestKnnGrid.predict(test_data[columnasRelevantes])

In [ ]:
print("Test Metrics:")
print(classification_report(test_data["Churn Label"], y_pred_test))

# Imprimir la matriz de confusión para el conjunto de test
print("Confusion Matrix - Test:")
print(confusion_matrix(test_data["Churn Label"], y_pred_test))

In [ ]:
### En todas las preccion es muy alta

## Regresion Logistica

In [ ]:
## SHARON

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Crear y entrenar el modelo
regresion_logistica = LogisticRegression(penalty='l2', C=50, solver='lbfgs', max_iter=250, class_weight='balanced', multi_class='ovr')
regresion_logistica.fit(X_train, y_train)


In [ ]:
# Realizar predicciones en el conjunto de entrenamiento
y_train_pred = regresion_logistica.predict(X_train)

# Evaluar el modelo en el conjunto de entrenamiento
print("Accuracy en el conjunto de entrenamiento:", accuracy_score(y_train, y_train_pred))
print("Classification Report:\n", classification_report(y_train, y_train_pred))

In [ ]:
# Matriz de confusión
cm_train_log = confusion_matrix(y_train, y_train_pred)
print("Matriz de confusión en el conjunto de entrenamiento:\n", cm_train_log)

In [ ]:
# Realizar predicciones en el conjunto de validación
y_val_pred = regresion_logistica.predict(X_val)

# Evaluar el modelo
print("Accuracy en el conjunto de validación:", accuracy_score(y_val, y_val_pred))
print("Classification Report:\n", classification_report(y_val, y_val_pred))

In [ ]:
# Matriz de confusión
cm_log = confusion_matrix(y_val, y_val_pred)
print("Matriz de confusión validación:\n", cm_log)

In [ ]:
# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Conjunto de Entrenamiento')
plt.show()

## SVC

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
#Excluir a Churn Label
x_train = train_data.drop(columns=['Churn Label'])

In [ ]:
# Verificar características en X_train
print(f"Número de características en x_train: {x_train.shape[1]}")

In [ ]:
x_test = test_data.drop(columns=['Churn Label'])

In [ ]:
y_test = test_data["Churn Label"]

In [ ]:
x_val = val_data.drop(columns=['Churn Label'])

In [ ]:
y_val = val_data["Churn Label"]

##### AJUSTES SENCILLOS

In [ ]:
Modelo3SVC = SVC()

In [ ]:
#Entrenando
Modelo3SVC.fit(x_train, train_data["Churn Label"])

In [ ]:
y_pred_no_balance_train = Modelo3SVC.predict(x_train)

In [ ]:
print(classification_report(y_train, y_pred_no_balance_train))

In [ ]:
#Predicción
y_pred_no_balance = Modelo3SVC.predict(x_val)

In [ ]:
print(classification_report(y_val, y_pred_no_balance))

In [ ]:
# Al tener un accuracy alto veamos si está prediciendo sobre la clase mayoritaria
class_data = train_data['Churn Label'].value_counts()
class_majority = class_data.idxmax()
print(f'La clase mayoritaria es: {class_majority}')

In [ ]:
# Calcular el porcentaje de clase
class_percentage = train_data['Churn Label'].value_counts(normalize=True) * 100
print(class_percentage)

In [ ]:
#Está prediciendo sobre la clase 0 que es que el cliente no abandona el servicio con un 73.46 %, mientras que la clase 
#minoritaria es 26.53%
# Con 100% de predicción, se muestra como un modelo perfecto; que no hay falsos positivos. Pero tiende a la confusión si 
# las predicciones positivas son bajas.
# Con un 24.91% el modelo no dectecto los falsos negativos porque tien un recall de 75% aprox.

##### RESCALAR LAS CLASES

In [ ]:
#Creando pipeline
Modelo3SVC_ = make_pipeline(StandardScaler(), SVC(class_weight="balanced", gamma='auto'))

In [ ]:
#Para el problema de desequilibrio se está usando "balanced" que calcula los pesos inversamente proporcionales a la frecuencia
#de la clase, de está forma no cambiamos el conjunto de datos lo que ajusta el balance de errores
#a diferencia del submuestreo
#de está forma debería mejorar su recall

In [ ]:
#Ajustar el modelo al entrenamiento
Modelo3SVC_.fit(x_train, train_data["Churn Label"])

In [ ]:
y_pred = Modelo3SVC_.predict(x_train)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
#Predicción
y_pred_balance = Modelo3SVC_.predict(x_val)

In [ ]:
print(classification_report(y_val, y_pred_balance))

In [ ]:
# 95.13% son las veces que el modelo indicará que el cliente abandonará el servicio, para los falsos positivos
# el modelo comete pocos errores

# 90.39% es capaz de identificar la mayoría de los clientes que abandonará el servicio.

# Lo que indica que no hay muchos errores en la clasificación, al tener una alta precisión se puede ofrecer a los clientes
# algún beneficio para que no abandonen
# Y un buen recall se tendrán resultados significativos que que clientes están en riesgo de abandonar el servicio lo que si
# me permite focalizar los esfuerzos que se tienen que hacer en la retención.

## Random Forest

In [ ]:
## Michael Poveda

#### Without grid and all columns

In [ ]:
X_train = train_data.drop(columns=["Churn Label"])
y_train = train_data["Churn Label"]

X_val = val_data.drop(columns=["Churn Label"])
y_val = val_data["Churn Label"]

X_test = test_data.drop(columns=["Churn Label"])
y_test = test_data["Churn Label"]

In [ ]:
KNclassifier1 = KNeighborsClassifier(n_neighbors=20)

In [ ]:
KNclassifier1.fit(X_train[columnasModelo], y_train)

In [ ]:
train_metrics_kn = train_data.copy()
train_metrics_kn["Preciction_knn1"] = KNclassifier1.predict(train_data[columnasModelo]) 

val_metrics_kn = val_data.copy()
val_metrics_kn["Preciction_knn1"] = KNclassifier1.predict(val_data[columnasModelo])



In [ ]:
print("Training Metrics:")
print(classification_report(y_true=train_metrics_kn["Churn Label"], y_pred=train_metrics_kn["Preciction_knn1"]))

In [ ]:
print("Training Metrics:")
print(classification_report(y_true=val_metrics_kn["Churn Label"], y_pred=val_metrics_kn["Preciction_knn1"]))

In [ ]:
cm = confusion_matrix(val_metrics_kn["Churn Label"], val_metrics_kn["Preciction_knn1"])
print(cm)
# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
#### no completo aun

In [ ]:
search.fit(X=train_data.drop(columns=["Target"]), y=train_data["Target"])

In [ ]:
best_model = search.best_estimator_

In [ ]:
best_model

In [ ]:
search.best_params_

In [ ]:
search_results = pd.DataFrame(search.cv_results_)

In [ ]:
search_results

# 6. Model performance evaluation

In [ ]:
y_pred = best_model.predict(val_data.drop(columns=["Target"]))
y_pred_prob = best_model.predict_proba(val_data.drop(columns=["Target"]))[:, 1]

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true=val_data["Target"], y_pred=y_pred)

In [ ]:
print(classification_report(y_true=val_data["Target"], y_pred=y_pred))

# 7. Explainability

In [ ]:
explainer = shap.KernelExplainer(bestKnnGrid.predict, train_data[columnasRelevantes].sample(n=100, random_state=1))

In [ ]:
explanation = explainer.shap_values(val_data[columnasRelevantes])

In [ ]:
plt.figure(figsize=(20,8))
shap.summary_plot(explanation.values[:,:,1], features=train_data.drop(columns=["Target"]), plot_type="bar")

In [ ]:
plt.figure(figsize=(20,8))
shap.summary_plot(explanation.values[:,:,1], features=train_data.drop(columns=["Target"]))

In [ ]:
shap.plots.waterfall(explanation[6,:,1], max_display=20)